# Rubin (DP0) & Roman (Troxel+23) images

Contact: Melissa Graham

**Based on a tutorial by Chien-Hao Lin.**

Date: Mon Nov 11 2024

RSP Image: Weekly 2024_42

Goal: Recreate Chien-Hao's notebook and use Roman images to help with deblending Rubin images.

## Introduction

Space-based images have much higher resolution.
Stars and galaxies that are very close together or even overlapping (blended) due to chance
alignments along the line-of-sight can be better distinguished in higher resolution images.
So can actual galaxy mergers in close physical proximity, though this is less common.

It is possible to use the locations of physically distinct objects (deblended objects)
from higher resolution images to make more accurate photometry measurements.
In cases where the higher and lower resolution images are obtained in the same filters,
and with similar depths, it makes sense just to use the higher resolution images alone.

However, Rubin will obtain data in optical filters and Roman in infrared filters.
In this case, using the higher-resolution infrared images to determine the number and
location of distinct objects, and then make photometric measurements in Rubin's
optical-range images, can improve the optical photometric measurements.

Roman DC2 Simulated Images and Catalogs at IRSA IPAC:<br>
https://irsa.ipac.caltech.edu/data/theory/Roman/Troxel2023/overview.html

Troxel et al. (2023):<br>
https://academic.oup.com/mnras/article/522/2/2801/7076879?login=false

## Set up

In [ ]:
import numpy as np

import lsst.geom
import lsst.afw.image as afwImage
import lsst.afw.display as afwDisplay
from lsst.daf.butler import Butler as dafButler

import lsst.daf.base as dafBase
from lsst.pipe.tasks.characterizeImage import CharacterizeImageTask
from lsst.meas.algorithms.detection import SourceDetectionTask
from lsst.meas.deblender import SourceDeblendTask
from lsst.meas.base import SingleFrameMeasurementTask
import lsst.afw.table as afwTable

Set the display backed to be Firefly.

In [ ]:
afwDisplay.setDefaultBackend('firefly')

Instantiate the butler.

In [ ]:
butler = dafButler('dp02', collections='2.2i/runs/DP0.2')

Define `ra` and `dec`, the central coordinates of interest.

Define the scale, in arcseconds per pixel, of Rubin and Roman images.

Define the stamp size to use when visualizing the images (i.e., the cutout size), in Rubin pixels; then use `stampsize / scale_ratio` as the extent when visualizing Roman images.

In [ ]:
ra, dec = 54.28, -38.30
rubin_scale = 0.2
roman_scale = 0.0575
stampsize = 150
scale_ratio = rubin_scale/roman_scale

Convert the coordinates to type `SpherePoint`.

In [ ]:
radec = lsst.geom.SpherePoint(ra, dec, lsst.geom.degrees)

For Rubin images: identify Rubin DP0.2 butler tract and patch for the desired coordinates.

In [ ]:
skymap = butler.get('skyMap')
tract = skymap.findTract(radec).tract_id
patch = skymap.findTract(radec).findPatch(radec).getSequentialIndex()
print(tract, patch)

For Roman images: four filters for one patch of deeply coadded Roman images have been stored in the shared space in the `/project` directory.

In [ ]:
ro_img_path = '/project/melissagraham2/troxel2023/'
ro_img_fnms = ['dc2_Y106_54.24_-38.3.fits',
               'dc2_J129_54.24_-38.3.fits',
               'dc2_H158_54.24_-38.3.fits',
               'dc2_F184_54.24_-38.3.fits']

## Visualize images

### Rubin images

For each filter, retrieve the `deepCoadd` patch and display the cutout in Firefly frames 1 through 6.

In [ ]:
lsst_bands = ['u', 'g', 'r', 'i', 'z', 'y']
for i in range(6):
    print(i+1, lsst_bands[i])
    afw_display = afwDisplay.Display(frame=i+1)
    dataId = {'tract': tract, 'patch': patch, 'band': lsst_bands[i]}
    image = butler.get('deepCoadd', dataId=dataId)
    size = stampsize
    extent = lsst.geom.ExtentI(size, size)
    cutout = image.getCutout(radec, extent)
    afw_display.mtv(cutout)
    afw_display.setMaskTransparency(100)
    del dataId, image, size, extent, cutout

### Roman images

Load and display a small cutout from each of the four images in Firefly frames 7 through 10.

> **Warnings:** Below, the warnings about unreadable mask extensions can be disregarded for the purposes of this tutorial, but generally when using the LSST Science Pipelines with non-Rubin data, all warnings should be followed up and third-party data sets might need to be reformatted to work properly.
In this case the images have four extensions: SCI, WHT, CTX, ERR.
But the `readFits` function expects MASK and IMAGE extensions.

In [ ]:
for i in range(4):
    afw_display = afwDisplay.Display(frame=i+7)
    fnm = ro_img_path + ro_img_fnms[i]
    print(i+7, fnm)
    image = afwImage.ExposureF.readFits(fnm)
    size = stampsize * scale_ratio
    extent = lsst.geom.ExtentI(size, size)
    cutout = image.getCutout(radec, extent)
    afw_display.mtv(cutout)
    del fnm, image, size, extent, cutout

### Clean up the Firefly window

Close each panel by clicking on the 'X' in the upper right corner until the display again says "Firefly Ready" and "Awaiting Python API Commands".

## Visualize detected sources

### Rubin

Source detection has already been run on the Rubin DP0 images.

The threshold is 5-sigma.

Display the r-band image cutout and overplot detected sources.

In [ ]:
dataId = {'tract': tract, 'patch': patch, 'band': 'r'}

In [ ]:
objects = butler.get('objectTable', dataId=dataId)

In [ ]:
# objects

Extract coordinate data into `numpy` arrays.

In [ ]:
ra_vals = np.asarray(objects['coord_ra'], dtype='float')
dec_vals = np.asarray(objects['coord_dec'], dtype='float')
x_vals = np.asarray(objects['x'], dtype='float')
y_vals = np.asarray(objects['y'], dtype='float')

Display the r-band cutout.

In [ ]:
afw_display = afwDisplay.Display(frame=1)
image = butler.get('deepCoadd', dataId=dataId)
size = stampsize
extent = lsst.geom.ExtentI(size, size)
cutout = image.getCutout(radec, extent)
afw_display.mtv(cutout)
afw_display.setMaskTransparency(100)

Mark detected objects with orange circles.

The stampsize of 150 x 150 pixels is 30 x 30 arcsec, or about 0.008 x 0.008 degrees.
Only plot objects thata re in the cutout.

In [ ]:
tx = np.where((ra_vals > ra - 0.01) &
              (ra_vals < ra + 0.01) &
              (dec_vals > dec - 0.01) &
              (dec_vals < dec + 0.01))[0]
print(len(tx))

In [ ]:
with afw_display.Buffering():
    for x in tx:
        afw_display.dot('o', x_vals[x], y_vals[x],
                        size=5, ctype='orange')

Clean up.

In [ ]:
del ra_vals, dec_vals, x_vals, y_vals
del tx
del dataId, image, size, extent, cutout

### Roman

#### Code straight from other tutorial

Ends up failing, try to fix later.

In [ ]:
# config_detection = SourceDetectionTask.ConfigClass()
# config_deblend = SourceDeblendTask.ConfigClass()
# config_meas = SingleFrameMeasurementTask.ConfigClass() 
# config_deblend.propagateAllPeaks = True
# config_deblend.maskPlanes=[]
# schema = afwTable.SourceTable.makeMinimalSchema()
# detectionTask = SourceDetectionTask(schema=schema, config=config_detection)
# sourceDeblendTask = SourceDeblendTask(schema=schema, config=config_deblend)
# measureTask = SingleFrameMeasurementTask(schema=schema, config=config_meas)

Something happened this class lost its name.

In [ ]:
# def (self, image):
#     exp = image.exp
#     tab = afwTable.SourceTable.make(self.schema)
#     ## Note that exp will be modified after running detection (calexp)
#     detections = self.detectionTask.run(tab, exp, doSmooth=self.detection_dosmooth, sigma=self.detection_sigma)
#     sources = detections.sources

#     self.sourceDeblendTask.run(exp, sources) ##exp is now calexp
#     self.measureTask.measure(sources, exp)

#     return sources, detections

In [ ]:
# fnm = ro_img_path + ro_img_fnms[2]
# print(fnm)
# image = afwImage.ExposureF.readFits(fnm)
# size = stampsize * scale_ratio
# extent = lsst.geom.ExtentI(size, size)
# cutout = image.getCutout(radec, extent)
# del fnm, image, size, extent

In [ ]:
# exp_H = cutout
# tab_H = afwTable.SourceTable.make(schema)

# detections_H = detectionTask.run(tab_H, exp_H,
#                                  doSmooth=True, sigma=None)
# sources_H = detections_H.sources
# sourceDeblendTask.run(exp_H, sources_H)
# measureTask.measure(sources_H, exp_H)

#### Use DP0.2 tutorial 05 as a guide

THIS ALSO, ULTIMATELY, FAILS.

Figure it out later.

Start by setting up the configurations for each task.

In [ ]:
config_charaterize = CharacterizeImageTask.ConfigClass()
config_charaterize.psfIterations = 1

config_detection = SourceDetectionTask.ConfigClass()
config_detection.thresholdValue = 5
config_detection.thresholdType = "stdev"

config_deblend = SourceDeblendTask.ConfigClass()
config_deblend.propagateAllPeaks = True
config_deblend.maskPlanes=[]

config_measure = SingleFrameMeasurementTask.ConfigClass() 

In [ ]:
schema = afwTable.SourceTable.makeMinimalSchema()
raerr = schema.addField("coord_raErr", type="F")
decerr = schema.addField("coord_decErr", type="F")

algMetadata = dafBase.PropertyList()

Define the tasks.

In [ ]:
characterizeTask = CharacterizeImageTask(config=config_charaterize)
detectionTask = SourceDetectionTask(schema=schema, config=config_detection)
deblendTask = SourceDeblendTask(schema=schema, config=config_deblend)
measureTask = SingleFrameMeasurementTask(schema=schema,
                                         config=config_measure,
                                         algMetadata=algMetadata)

Get the Roman image and make a cutout.

In [ ]:
fnm = ro_img_path + ro_img_fnms[2]
print(fnm)

In [ ]:
image = afwImage.ExposureF.readFits(fnm)
size = stampsize * scale_ratio
extent = lsst.geom.ExtentI(size, size)
cutout = image.getCutout(radec, extent)

In [ ]:
tab = afwTable.SourceTable.make(schema)

The following run to characterize the image does get a pink warning but it does not seem fatal.

In [ ]:
result = characterizeTask.run(image)

In [ ]:
result = detectionTask.run(tab, image)

In [ ]:
result.numPosPeaks

In [ ]:
sources = result.sources

In [ ]:
# sources

In [ ]:
deblendTask.run(image, sources)

In [ ]:
measureTask.run(measCat=sources, exposure=image)

In [ ]:
sources = sources.copy(True)

Display the Roman H-band cutout in frame 2.

In [ ]:
afw_display = afwDisplay.Display(frame=2)
afw_display.mtv(cutout)
afw_display.setMaskTransparency(100)

Overplot detected sources.

Right away notice something bad has happened with the source coordinates.

In [ ]:
# sources.asAstropy()

In [ ]:
ra_vals = np.asarray(sources['coord_ra'], dtype='float')
dec_vals = np.asarray(sources['coord_dec'], dtype='float')
x_vals = np.asarray(sources.getX(), dtype='float')
y_vals = np.asarray(sources.getY(), dtype='float')

In [ ]:
print(np.min(ra_vals), np.max(ra_vals))
print(np.min(dec_vals), np.max(dec_vals))
print(np.min(x_vals), np.max(x_vals))
print(np.min(y_vals), np.max(y_vals))

In [ ]:
extent

In [ ]:
cutout.image.getBBox()

In [ ]:
tx = np.where((x_vals > 2303) &
              (x_vals < 5384) &
              (y_vals > 2303) &
              (y_vals < 5384))[0]
# tx = np.where((ra_vals > ra - 0.01) &
#               (ra_vals < ra + 0.01) &
#               (dec_vals > dec - 0.01) &
#               (dec_vals < dec + 0.01))[0]
print(len(tx))

Try to plot them anyway. This fails.

In [ ]:
with afw_display.Buffering():
    for x in tx:
        afw_display.dot('o', x_vals[x], y_vals[x],
                        size=20, ctype='orange')

Clean up.

In [ ]:
del tx, ra_vals, dec_vals, x_vals, y_vals
del fnm, image, size, cutout, extent
del tab, result, sources

In [ ]:
del config_characterize, config_detection, config_debelnd, config_measure
del algMetadata, schema
del characterizeTask, detectionTask, deblendTask, measureTask